In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Text
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

In [2]:
setting = 4

In [23]:
if setting == 1 or setting == 2:    
    df = pd.read_csv('./raw/ids_2020-06.csv', header=None)
    df.columns = ['submission_id', 'subreddit_id', 'created_utc', 'subreddit', 'url']
    print(len(df))

    df2 = pd.read_csv('./raw/ids_2020-07.csv', header=None)
    df2.columns = ['submission_id', 'subreddit_id', 'created_utc', 'subreddit', 'url']
    print(len(df2))
    
    all_df = pd.concat([df, df2])
    print(len(all_df))
if setting >= 3:
    all_df = pd.read_csv('./raw/ids_2022-12.csv', header=None)
    all_df.columns = ['submission_id', 'subreddit_id', 'created_utc', 'subreddit', 'url']
    print(len(all_df))

In [22]:
if setting == 1:
    db_name = 'before:politics.db'
    subreddit = 'politics'
    seed = 0
    # Mon Jun 08 2020 07:00:00 GMT+0000
    after_utc = 1591599600
    # Mon Jun 29 2020 07:00:00 GMT+0000
    before_utc = 1593414000
elif setting == 2:
    db_name = 'after:politics.db'
    subreddit = 'politics'
    seed = 0
    # Mon Jun 29 2020 07:00:00 GMT+0000
    after_utc = 1593414000
    # Sun Jul 20 2020 07:00:00 GMT+0000
    before_utc = 1595228400
elif setting == 3:
    db_name = 'askreddit.db'
    subreddit = 'AskReddit'
    seed = 0
    after_utc = None
    before_utc = None

In [24]:
df = all_df

if subreddit:
    df = df[df.subreddit == subreddit]
    print(len(df)) 
    
df = df.drop_duplicates('submission_id')
print(len(df)) 

if before_utc:
    df = df[df.created_utc < before_utc]
    print(len(df))
    
if after_utc:
    df = df[df.created_utc > after_utc]
    print(len(df)) 
    
df

15978
15978


,submission_id,subreddit_id,created_utc,subreddit,url
883,z97vv4,t5_2rawz,1669852837,legaladvice,https://www.reddit.com/r/legaladvice/comments/...
1856,z97wv9,t5_2rawz,1669852899,legaladvice,https://www.reddit.com/r/legaladvice/comments/...
2521,z97xk3,t5_2rawz,1669852944,legaladvice,https://www.reddit.com/r/legaladvice/comments/...
2543,z97xkz,t5_2rawz,1669852946,legaladvice,NaN
4228,z97zdd,t5_2rawz,1669853065,legaladvice,https://www.reddit.com/r/legaladvice/comments/...
...,...,...,...,...,...
35666517,1006jp5,t5_2rawz,1672529409,legaladvice,https://www.reddit.com/r/legaladvice/comments/...
35673954,1006q9j,t5_2rawz,1672529981,legaladvice,https://www.reddit.com/r/legaladvice/comments/...
35676630,1006sn3,t5_2rawz,1672530186,legaladvice,https://www.reddit.com/r/legaladvice/comments/...
35676766,1006srm,t5_2rawz,1672530196,legaladvice,https://www.reddit.com/r/legaladvice/comments/...


In [25]:
# create submissions database
engine = create_engine('sqlite:///%s' % db_name, echo = False)
meta = MetaData()

submissions_table = Table(
   'submissions', meta, 
    Column('id', String, primary_key=True),
    Column('status', Integer),
)
meta.create_all(engine)

In [26]:
session = Session(engine)

with session.begin():
    for i, row in tqdm(df.sample(frac=1, random_state=seed).iterrows(), total=len(df)):
        ins = submissions_table.insert().values(id=row['submission_id'], status=0)
        session.execute(ins)

  0%|          | 0/15978 [00:00<?, ?it/s]

In [27]:
# create comments database
engine = create_engine('sqlite:///%s' % db_name, echo = True)
meta = MetaData()

students = Table(
   'comments', meta, 
    Column('comment_id', String, primary_key=True),
    Column('submission_id', String),
    Column('url', String),
    Column('comment', Text),    
)
meta.create_all(engine)

2023-01-21 02:51:48,378 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-21 02:51:48,379 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("comments")
2023-01-21 02:51:48,380 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-21 02:51:48,398 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("comments")
2023-01-21 02:51:48,400 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-21 02:51:48,402 INFO sqlalchemy.engine.Engine 
CREATE TABLE comments (
	comment_id VARCHAR NOT NULL, 
	submission_id VARCHAR, 
	url VARCHAR, 
	comment TEXT, 
	PRIMARY KEY (comment_id)
)


2023-01-21 02:51:48,403 INFO sqlalchemy.engine.Engine [no key 0.00138s] ()
2023-01-21 02:51:48,425 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
df.head(1)

,submission_id,subreddit_id,created_utc,subreddit,url
883,z97vv4,t5_2rawz,1669852837,legaladvice,https://www.reddit.com/r/legaladvice/comments/...


In [29]:
counts = pd.DataFrame(df['subreddit'].value_counts())

In [30]:
for i, j in counts.head(100).iterrows():
    print(i, j.iloc[0])

legaladvice 15978
